In [ ]:
!pip install langchain
!pip install huggingface_hub
!pip install pypdf
!pip install faiss-cpu
!pip install sentence-transformers
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.2 MB/s eta 0:00:00


In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate
from huggingface_hub import login

**Hugging Face token login methods (Fine-Grained)**

In [ ]:
import os

#Edwin's Token
# os.environ['HUGGINGFACE_TOKEN'] = [token]
# os.environ['HUGGINGFACEHUB_API_TOKEN'] = [token]

In [ ]:
# Load the PDF document

from langchain.document_loaders import PyPDFLoader

pdfloader = PyPDFLoader("/content/drive/MyDrive/Colab Notebooks/RAG/RAG-data/Fostering_a_Positive_Workplace_Culture_Impacts_on_.pdf")
documents = pdfloader.load()

In [ ]:
len(documents)

18

In [ ]:
# Split the PDF into manageable chunks

from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

In [ ]:
# Load the embedding model

from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

<ipython-input-9-7472a9e7e8f7>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Create a FAISS vector store for efficient retrieval & a retriver

from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, embedding_model)

retriever = db.as_retriever()

In [ ]:
# Load the google/flan-t5-base model

from langchain.llms import HuggingFaceHub

repo_id = "google/flan-t5-base" # Changed to a smaller model
llm = HuggingFaceHub(
    repo_id=repo_id,
    model_kwargs={"temperature": 0.7, "max_length": 128}
)

<ipython-input-11-3521a9fea76a>:6: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceHub(


In [ ]:
llm.invoke("How does communication impact the workplace culture?")

'communication is a key component of the workplace culture.'

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = """Based on the provided context, respond to the question below while adhering to these guidelines:
1. If the answer is unclear or not found, do not speculate. Instead, state, "I do not know the answer"
2. If the answer is found, provide a clear and concise response in no more than ten sentences.


Question: {question}

Answer:

{context}
"""


PROMPT = PromptTemplate(
 template=prompt_template, input_variables=["context", "question"]
)

In [ ]:
# Set up the retrieval-based QA chain

from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

In [ ]:
# Test the QA system with a question

query = "How does communication impact the workplace culture?"
result = qa_chain.invoke({"query": query})

print(f"Answer: {result['result']}")

Answer: The concept of a positive workplace culture is a common one.


In [ ]:
# Test the QA system with a question

query2 = "What are the benefits of creating a positive workplace culture for an organization?"
result2 = qa_chain.invoke({"query": query2})

print(f"Answer: {result2['result']}")

Answer: [43]


In [ ]:
# Print source documents

print("\nSource Documents:")
for doc in result['source_documents']:
    print(doc.page_content[:200])  # Print a snippet of the source


Source Documents:
Human Resource Management - An Update
2
development, and collaborative dynamics, organizations can unlock the latent 
capabilities within their workforce, resulting in elevated levels of performance a
Human Resource Management - An Update
4
2.2 Crafting the essence: deconstructing the positive workplace culture
Defining a positive organizational culture is not a one-size-fits-all task, as 
differen
1
Chapter
Fostering a Positive Workplace 
Culture: Impacts on Performance 
and Agility
Cătălina Radu
Abstract
This chapter explores the profound influence of a positive workplace culture on 
work perf
Fostering a Positive Workplace Culture: Impacts on Performance and Agility
DOI: http://dx.doi.org/10.5772/intechopen.1003259
15
and organizational commitment on 
workforce sustainability. Sustainabili
